In [10]:
import numpy as np
from scipy import interpolate
from gurobipy import *
#import matplotlib.pyplot as plt
#def solve(SOCval):
m = Model("linearinterp")
Eint = [10, 20] # Initial Energy in EV
Ereq = [80, 60]
te = [28.0, 24.0]
SOC = [0,50,100]    #SOC level in %
Energy = [0,20,40]  #Energy in kwh
f = interpolate.interp1d(SOC,Energy)
x = Eint[0] #Energy30
y = Ereq[0] #R_E
Energy30 = f(x)
R_E = f(y)

print(Energy30)
print(type(SOC))
print(type(Energy))
print(type(Energy30))
print(type(R_E))
print('type of Energy30',Energy30,type(Energy30))
print('type of R_E',R_E,type(R_E))
m.addConstr(x + te[0] == y, "U_c")
m.update()

4.0
<class 'list'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
type of Energy30 4.0 <class 'numpy.ndarray'>
type of R_E 32.0 <class 'numpy.ndarray'>


In [3]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-
import numpy as np
from scipy import interpolate
from gurobipy import *

#Create optimization model
m = Model('minimizecost')

# Model data and Variable assignment

M = 2           # Number of EV's
T = 10          # Total time slot
TS1 = 0         # Start time of EV1
TD1 = 5         # End time of EV1
TS2 = 4         # Start time of EV2
TD2 = 10        # End time of EV1
eff = 0.95      # Efficiency
ft = 15
Eint = [20, 20] # Initial Energy in EV
Ereq = [80, 60] # Required Energy in EV by user
cost = [0.10, 0.20, 0.60, 0.40, 0.40, 0.30, 0.20, 0.20, 0.80, 0.60]

C = []
e = []
pa =[[0 for t in range(T)] for i in range(M)]
pb =[[0 for t in range(T)] for i in range(M)]
pc =[[0 for t in range(T)] for i in range(M)]
patot =[0 for t in range(T)]
pbtot =[0 for t in range(T)]
pctot =[0 for t in range(T)]
pab =[0 for t in range(T)]
pbc =[0 for t in range(T)]
pca =[0 for t in range(T)]
mc =[[0 for t in range(T)] for i in range(M)]
e =[[0 for t in range(T)] for i in range(M)]
subcost =[0 for i in range(M)]
te =[0 for t in range(T)]

# Create variables

l= list([(i,t) for i in range(M) for t in range(T)])
a = m.addVars(l, vtype=GRB.BINARY,name="a")
b = m.addVars(l, vtype=GRB.BINARY,name="b")
c = m.addVars(l, vtype=GRB.BINARY,name="c")
p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")


#Phase constraint
for i in range(M):
    for t in range(T):
        m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint")
    
#SOC user constraint
for t in range(TS1,TD1):
    for i in range(M):
        if i == 0:
            e[i][t] += p[i,t] * eff   # Energy consumed by EV1   ft *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            #print(e[i][t])
            subcost[i] += mc[0][t]
            #print(subcost[i])
            te[i] += e[0][t]
            n = te[i]
SOC = [0,50,100]    #SOC level in %
Energy = [0,20,40]  #Energy in kwh
f = interpolate.interp1d(SOC,Energy)
x = Eint[0] 
y = Ereq[0] 
I_E = f(x)
R_E = f(y)
I_E = int(np.round(I_E))
R_E = int(np.round(R_E))
m.addConstr(n + I_E == R_E, "User_constraint")
for t in range(TS2,TD2):
    for i in range(M):
        if i == 1:
            e[i][t] += p[i,t] * eff   # Energy consumed by EV2 ft *
            mc[i][t] = e[i][t] * cost[t]      # Cost for each EV
            #print(e[i][t])
            subcost[i] += mc[1][t]
            #print(subcost[i])
            te[i] += e[1][t]
            n = te[i]
SOC = [0,50,100]    #SOC level in %
Energy = [0,20,40]  #Energy in kwh
f = interpolate.interp1d(SOC,Energy)
x = Eint[1] 
y = Ereq[1] 
I_E = f(x)
R_E = f(y)
I_E = int(np.round(I_E))
R_E = int(np.round(R_E)) 
m.addConstr(n + I_E == R_E, "User_constraint")

#Grid constraint
for t in range(T):
    for i in range(M):
    #if i < M :
        pa[i][t] = a[i,t] * p[i,t]
        pb[i][t] = b[i,t] * p[i,t]
        pc[i][t] = c[i,t] * p[i,t]
        #print(pa[i][t])
    patot[t] = pa[0][t] + pa[1][t]
    pbtot[t] = pb[0][t] + pb[1][t]
    pctot[t] = pc[0][t] + pc[1][t]
   
    pab[t] = patot[t] - pbtot[t]
    pbc[t] = pbtot[t] - pctot[t]
    pca[t] = pctot[t] - patot[t]
    
    m.addConstr(pab[t] <= 4.6, name="Power_A_B_constraint")
    m.addConstr(pab[t] >= -4.6, name="Power_A_B_constraint")
    m.addConstr(pbc[t] <= 4.6, name="Power_B_C_constraint")
    m.addConstr(pbc[t] >= -4.6, name="Power_B_C_constraint")
    m.addConstr(pca[t] <= 4.6, name="Power_C_A_constraint")
    m.addConstr(pca[t] >= -4.6, name="Power_C_A_constraint")
 #power outside TS TD as 0
for i in range(M):
    if i == 0:
        for t in range(0,TS1) :
            m.addConstr(p[i,t] == 0, "Power_constraint")
        for t in range(TD1,T) :
            m.addConstr(p[i,t] == 0, "Power_constraint")

for i in range(M):
    if i == 1:
        for t in range(0,TS2) : 
            m.addConstr(p[i,t] == 0, "Power_constraint")
        for t in range(TD2,T) :
            m.addConstr(p[i,t] == 0, "Power_constraint")
m.update()   
    
#set objective:
m.ModelSense = GRB.MINIMIZE
C = sum(subcost[i] for i in range(M))
m.setObjective(C)
m.write('mincost.lp')

# Compute optimal solution
m.optimize()

#print solution 
for v in m.getVars():
    print(v.varName, v.x)
    
print('Obj:', m.objVal)


#"""
# do IIS
#print('The model is infeasible; computing IIS')
#m.computeIIS()
#if m.IISMinimal:
#  print('IIS is minimal\n')
#else:
#  print('IIS is not minimal\n')
#print('\nThe following constraint(s) cannot be satisfied:')
#for c in m.getConstrs():
#    if c.IISConstr:
#         print('%s' % c.constrName)
 #if te[i] + I_E == R_E :
             #   return true
            #else:
             #   return false
           # return I_E
           # return R_E
           # print(type(I_E))
            #print(type(R_E))

Optimize a model with 31 rows, 80 columns and 80 nonzeros
Model has 60 quadratic constraints
Variable types: 20 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [9e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e-01, 8e-01]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [5e+00, 5e+00]
Presolve removed 18 rows and 36 columns
Presolve time: 0.01s
Presolved: 271 rows, 176 columns, 770 nonzeros
Variable types: 143 continuous, 33 integer (33 binary)

Root relaxation: objective 4.848000e+00, 112 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.84800    0   26          -    4.84800      -     -    0s
H    0     0                      13.6460000    4.84800  64.5%     -    0s
     0     0    9.78063    0   14   13.64600    9.78063  28.3%     -    0s
     0     0  

In [49]:
#IMPLEMENTING WITH HOUSEHOLD
#!/usr/bin/env python
#-*- coding: utf-8 -*-
import csv
import numpy as np
from scipy import interpolate
from gurobipy import *

#Create optimization model
m = Model('minimizecost')

# Model data and Variable assignment

M = 2           # Number of EV's
T = 20          # Total time slot
TSD1 = [0, 9]         # Start time and End time of EV1

TSD2 = [6, 16]         # Start time End time of EV2

eff = 0.95      # Efficiency
ft = 15
Eint = [10, 10] # Initial Energy in EV
Ereq = [90, 90] # Required Energy in EV by user
#cost = [0.10, 0.20, 0.60, 0.40, 0.40, 0.30, 0.20, 0.20, 0.80, 0.60]

path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/household8_profile.csv"
file = open(path1, newline='')
reader = csv.reader(file)
header = next(reader)
HH = []
for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
    HHP_a = float(row[0])
    HHP_b = float(row[1])
    HHP_c = float(row[2])
    HHQ_a = float(row[3])
    HHQ_b = float(row[4])
    HHQ_c = float(row[5])
    HH.append([HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
file2 = open(path2, newline='')
reader2 = csv.reader(file2)
header2 = next(reader2)
marketcost = []
for row in reader2 :
    # row = ['Price [ct/kWh]']
    mcost = float(row[0])
    marketcost.append(mcost)
cost = [j/100 for j in marketcost]
path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile.csv"
file3 = open(path3, newline='')
reader3 = csv.reader(file3)
header3 = next(reader3)
PV =[]
for row in reader3 :
    # row = ['Power [kW]']
    PV_P = float(row[0])
    PV.append(PV_P)

var_names = []
C = []
e = []
pa =[[0 for t in range(T)] for i in range(M)]
pb =[[0 for t in range(T)] for i in range(M)]
pc =[[0 for t in range(T)] for i in range(M)]
patot =[0 for t in range(T)]
pbtot =[0 for t in range(T)]
pctot =[0 for t in range(T)]
pab =[0 for t in range(T)]
pbc =[0 for t in range(T)]
pca =[0 for t in range(T)]
mc =[[0 for t in range(T)] for i in range(M)]
e =[[0 for t in range(T)] for i in range(M)]
subcost =[0 for i in range(M)]
te =[0 for t in range(T)]

# Create variables

l= list([(i,t) for i in range(M) for t in range(T)])
a = m.addVars(l, vtype=GRB.BINARY,name="a")
b = m.addVars(l, vtype=GRB.BINARY,name="b")
c = m.addVars(l, vtype=GRB.BINARY,name="c")
p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")


#Phase constraint
for i in range(M):
    for t in range(T):
        m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint")
    
#SOC user constraint
for t in range(TSD1[0],TSD1[1]):
    for i in range(M):
        if i == 0:
            e[i][t] += p[i,t] * eff   # Energy consumed by EV1   ft *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[0][t]
            te[i] += e[0][t]
            n = te[i]
SOC = [0,50,100]    #SOC level in %
Energy = [0,20,40]  #Energy in kwh
f = interpolate.interp1d(SOC,Energy)
x = Eint[0] 
y = Ereq[0] 
I_E = f(x)
R_E = f(y)
I_E = int(np.round(I_E))
R_E = int(np.round(R_E))
m.addConstr(n + I_E == R_E, "User_constraint")
for t in range(TSD2[0],TSD2[1]):
    for i in range(M):
        if i == 1:
            e[i][t] += p[i,t] * eff   # Energy consumed by EV2 ft *
            mc[i][t] = e[i][t] * cost[t]      # Cost for each EV
            subcost[i] += mc[1][t]
            te[i] += e[1][t]
            n = te[i]
SOC = [0,50,100]    #SOC level in %
Energy = [0,20,40]  #Energy in kwh
f = interpolate.interp1d(SOC,Energy)
x = Eint[1] 
y = Ereq[1] 
I_E = f(x)
R_E = f(y)
I_E = int(np.round(I_E))
R_E = int(np.round(R_E)) 
m.addConstr(n + I_E == R_E, "User_constraint")

#Grid constraint
for t in range(T):
    for i in range(M):
    
        pa[i][t] = a[i,t] * (p[i,t] + HH[t][0] - PV[t])  
        pb[i][t] = b[i,t] * (p[i,t] + HH[t][1])  
        pc[i][t] = c[i,t] * (p[i,t] + HH[t][2])  
        
    patot[t] = pa[0][t] + pa[1][t]
    pbtot[t] = pb[0][t] + pb[1][t]
    pctot[t] = pc[0][t] + pc[1][t]
   
    pab[t] = patot[t] - pbtot[t]
    pbc[t] = pbtot[t] - pctot[t]
    pca[t] = pctot[t] - patot[t]
    
    m.addConstr(pab[t] <= 4.6, name="Power_A_B_constraint")
    m.addConstr(pab[t] >= -4.6, name="Power_A_B_constraint")
    m.addConstr(pbc[t] <= 4.6, name="Power_B_C_constraint")
    m.addConstr(pbc[t] >= -4.6, name="Power_B_C_constraint")
    m.addConstr(pca[t] <= 4.6, name="Power_C_A_constraint")
    m.addConstr(pca[t] >= -4.6, name="Power_C_A_constraint")
    
#power outside TS TD as 0
for i in range(M):
    if i == 0:
        for t in range(0,TSD1[0]) :
            m.addConstr(p[i,t] == 0, "Power_constraint")
        for t in range(TSD1[1],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint")

for i in range(M):
    if i == 1:
        for t in range(0,TSD2[0]) : 
            m.addConstr(p[i,t] == 0, "Power_constraint")
        for t in range(TSD2[1],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint")
m.update()   
    
#set objective:
m.ModelSense = GRB.MINIMIZE
C = sum(subcost[i] for i in range(M))
m.setObjective(C)
m.write('cost.lp')

# Compute optimal solution
m.optimize()

m.write('out.sol')

#print solution 

for v in m.getVars():
    print(v.varName, v.x)
    
print('Obj:', m.objVal)

    
with open('out.sol', newline='') as f:
    reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
    next(reader)
    next(reader)
    lines = list(reader)

with open('out.csv', 'w') as myfile:
    wr = csv.writer(myfile, delimiter=',')
    wr.writerows(lines)
    
f.close()
myfile.close()

        #print(row)
#"""
# do IIS
#print('The model is infeasible; computing IIS')
#m.computeIIS()
#if m.IISMinimal:
#  print('IIS is minimal\n')
#else:
#  print('IIS is not minimal\n')
#print('\nThe following constraint(s) cannot be satisfied:')
#for c in m.getConstrs():
#    if c.IISConstr:
#         print('%s' % c.constrName)


Optimize a model with 63 rows, 160 columns and 160 nonzeros
Model has 120 quadratic constraints
Variable types: 40 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [9e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e-03, 5e-01]
  Objective range  [3e-01, 3e-01]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 3e+01]
  QRHS range       [5e+00, 5e+00]
Presolve removed 29 rows and 45 columns
Presolve time: 0.01s
Presolved: 472 rows, 343 columns, 1521 nonzeros
Variable types: 247 continuous, 96 integer (96 binary)

Root relaxation: objective 1.843788e+01, 208 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   18.43788    0   26          -   18.43788      -     -    0s
H    0     0                      18.4820248   18.43788  0.24%     -    0s
H    0     0                      18.4729979   18

In [41]:
import csv
with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile_S.txt', newline='') as f:
    reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
    #next(reader)
    #next(reader)
    lines = list(reader)

with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile_Scheduling.csv', 'w', newline='') as myfile:
    wr = csv.writer(myfile, delimiter=',')
    wr.writerows(lines)

In [40]:
import csv
with open('C:/Users/Subha Thillairajan/PycharmProjects/mosaik/Envs/mosaik/src/mosaik-powerfacotry/demos/data/HH_profile_S.txt', newline='') as f:
    reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
    #next(reader)
    #next(reader)
    lines = list(reader)

with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH_profile_Scheduling.csv', 'w', newline='') as myfile:
    wr = csv.writer(myfile, delimiter=',')
    wr.writerows(lines)

In [2]:
#CSV FILE IMPORT TEST
import csv
path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH_profile_Scheduling.csv"
file = open(path1, newline='')
reader = csv.reader(file)
header = next(reader)
HH = []
for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
    HHDate = row[0]
    HHP_a = float(row[1])
    HHP_b = float(row[2])
    HHP_c = float(row[3])
    HHQ_a = float(row[4])
    HHQ_b = float(row[5])
    HHQ_c = float(row[6])
    HH.append([HHDate,HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
#data = [row for row in reader]
#print(header)
#print(len(HH))
for i in range(len(HH)):
    #print(HH[1][0])
    print(HH[i])
path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
file2 = open(path2, newline='')
reader2 = csv.reader(file2)
header2 = next(reader2)
cost = []
for row in reader2 :
    # row = ['Price [ct/kWh]']
    mcost = float(row[0])
    cost.append(mcost)
B = [j/100 for j in cost]
#print(B[0])
#print(header2)
#print(len(cost))
#print(cost)
path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile_Scheduling.csv"
file3 = open(path3, newline='')
reader3 = csv.reader(file3)
header3 = next(reader3)
PV =[]
for row in reader3 :
    # row = ['Price [ct/kWh]']
    PVDate = row[0]
    PV_P = float(row[1])
    PV.append([PVDate,PV_P])
print(header3)
print(len(PV))
print(PV[19][1])
#for i in range(len(PV)):
#    print(PV[i])

#for i in range(len(cost)):
    #x = [i / 100 for i in cost]    
    #print(cost[i])
#myInt = 100
#newList[:] = [x / myInt for x in cost]
#print(cost)


    #cost = []


['01.01.2014 00:00:00', 0.104, 0.088, 0.391, 0.0, -0.008, 0.021]
['01.01.2014 00:15:00', 0.145, 0.099, 0.134, -0.012, 0.073, 0.088]
['01.01.2014 00:30:00', 0.046, 0.096, 0.259, -0.024, -0.007, 0.051]
['01.01.2014 00:45:00', 0.148, 0.132, 0.121, 0.099, -0.052, 0.069]
['01.01.2014 01:00:00', 0.246, 0.004, 0.033, -0.003, -0.02, -0.005]
['01.01.2014 01:15:00', 0.089, 0.077, 0.182, 0.016, -0.047, 0.042]
['01.01.2014 01:30:00', 0.001, 0.122, 0.046, -0.009, 0.099, -0.023]
['01.01.2014 01:45:00', 0.052, 0.078, 0.032, 0.005, -0.073, 0.074]
['01.01.2014 02:00:00', 0.017, 0.067, 0.167, -0.016, 0.057, -0.01]
['01.01.2014 02:15:00', 0.009, 0.074, 0.027, -0.01, 0.027, -0.008]
['01.01.2014 02:30:00', 0.214, 0.074, 0.003, 0.094, 0.006, -0.001]
['01.01.2014 02:45:00', 0.05, 0.052, 0.097, -0.005, -0.003, 0.103]
['01.01.2014 03:00:00', 0.067, 0.05, 0.013, 0.001, 0.0, -0.001]
['01.01.2014 03:15:00', 0.088, 0.14, 0.025, 0.038, 0.003, 0.038]
['01.01.2014 03:30:00', 0.098, 0.054, 0.005, 0.039, -0.029, 0.001]

In [5]:
#IMPLEMENTING LOAD IMBALANCE OBJECTIVE
import numpy as np
from scipy import interpolate
from gurobipy import *
#import matplotlib.pyplot as plt
#def solve(SOCval):
m = Model("linearinterp")


M = 2
T = 10
TS1 = 0         # Start time of EV1
TD1 = 5         # End time of EV1
TS2 = 4         # Start time of EV2
TD2 = 10        # End time of EV1
eff = 0.95      # Efficiency
Eint = [50, 30] # Initial Energy in EV
Ereq = [80, 60]
cost = [0.10, 0.20, 0.60, 0.40, 0.40, 0.30, 0.20, 0.20, 0.80, 0.60]

Im_C = []
C = []
e = []
pa =[[0 for t in range(T)] for i in range(M)]
pb =[[0 for t in range(T)] for i in range(M)]
pc =[[0 for t in range(T)] for i in range(M)]
patot =[0 for t in range(T)]
pbtot =[0 for t in range(T)]
pctot =[0 for t in range(T)]
pab =[0 for t in range(T)]
pbc =[0 for t in range(T)]
pca =[0 for t in range(T)]
mc =[[0 for t in range(T)] for i in range(M)]
e =[[0 for t in range(T)] for i in range(M)]
subcost =[0 for i in range(M)]
Im_cost =[0 for i in range(M)]
te =[0 for t in range(T)]
#Im = [[0 for t in range(T)] for i in range(M)]
#x = [[0 for t in range(T)] for i in range(M)]
#Im_pbc = [0 for t in range(T)]
#Im_pca = [0 for t in range(T)]
# Create variables

l= list([(i,t) for i in range(M) for t in range(T)])
a = m.addVars(l, vtype=GRB.BINARY,name="a")
b = m.addVars(l, vtype=GRB.BINARY,name="b")
c = m.addVars(l, vtype=GRB.BINARY,name="c")
p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")


#Phase constraint
for i in range(M):
    for t in range(T):
        m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint")
    
#SOC user constraint
for t in range(TS1,TD1):
    for i in range(M):
        if i == 0:
            e[i][t] += p[i,t] * eff   # Energy consumed by EV1   ft *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            #print(e[i][t])
            subcost[i] += mc[0][t]
            
            #print(subcost[i])
            te[i] += e[0][t]
            n = te[i]
SOC = [0,50,100]    #SOC level in %
Energy = [0,20,40]  #Energy in kwh
f = interpolate.interp1d(SOC,Energy)
x = Eint[0] 
y = Ereq[0] 
I_E = f(x)
R_E = f(y)
I_E = int(np.round(I_E))
R_E = int(np.round(R_E))
m.addConstr(n + I_E == R_E, "User_constraint")
for t in range(TS2,TD2):
    for i in range(M):
        if i == 1:
            e[i][t] += p[i,t] * eff   # Energy consumed by EV2 ft *
            mc[i][t] = e[i][t] * cost[t]      # Cost for each EV
            #print(e[i][t])
            subcost[i] += mc[1][t]
           
            
            te[i] += e[1][t]
            n = te[i]
SOC = [0,50,100]    #SOC level in %
Energy = [0,20,40]  #Energy in kwh
f = interpolate.interp1d(SOC,Energy)
x = Eint[1] 
y = Ereq[1] 
I_E = f(x)
R_E = f(y)
I_E = int(np.round(I_E))
R_E = int(np.round(R_E)) 
m.addConstr(n + I_E == R_E, "User_constraint")

#Grid constraint
for t in range(T):
    for i in range(M):
    #if i < M :
        pa[i][t] = a[i,t] * p[i,t]
        pb[i][t] = b[i,t] * p[i,t]
        pc[i][t] = c[i,t] * p[i,t]
        #print(pa[i][t])
    patot[t] = pa[0][t] + pa[1][t]
    pbtot[t] = pb[0][t] + pb[1][t]
    pctot[t] = pc[0][t] + pc[1][t]
   
    pab[t] = patot[t] - pbtot[t]
    pbc[t] = pbtot[t] - pctot[t]
    pca[t] = pctot[t] - patot[t]
    
    p_a_b = max(pab)
    
    m.addConstr(pab[t] <= 2.3, name="Power_A_B_constraint")
    m.addConstr(pab[t] >= -4.6, name="Power_A_B_constraint")
    m.addConstr(pbc[t] <= 2.3, name="Power_B_C_constraint")
    m.addConstr(pbc[t] >= -4.6, name="Power_B_C_constraint")
    m.addConstr(pca[t] <= 2.3, name="Power_C_A_constraint")
    m.addConstr(pca[t] >= -4.6, name="Power_C_A_constraint")
    
for t in range(T):
    for i in range(M):
        if p_a_b <= 2.3:
            Im_cost[i] = subcost[i] + 1.0
        #elif pab[t] > 2.3:
        #    Im_cost[i] = subcost[i] + 1.0
        elif pbc[t] <= 2.3:
            Im_cost[i] = subcost[i] + 1.0
        #elif pbc[t] > 2.3:
        #    Im_cost[i] = subcost[i] + 1.0
        elif pca[t] <= 2.3:
            Im_cost[i] = subcost[i] + 1.0
        #elif pca[t] > 2.3:
        #    Im_cost[i] = subcost[i] + 1.0

#power outside TS TD as 0
for i in range(M):
    if i == 0:
        for t in range(0,TS1) :
            m.addConstr(p[i,t] == 0, "Power_constraint")
        for t in range(TD1,T) :
            m.addConstr(p[i,t] == 0, "Power_constraint")

for i in range(M):
    if i == 1:
        for t in range(0,TS2) : 
            m.addConstr(p[i,t] == 0, "Power_constraint")
        for t in range(TD2,T) :
            m.addConstr(p[i,t] == 0, "Power_constraint")
m.update()  


#set objective:
m.ModelSense = GRB.MINIMIZE
C = sum(subcost[i] for i in range(M))
Im_C = sum(Im_cost[i] for i in range(M))
m.setObjectiveN(C,0,2)
m.setObjectiveN(Im_C,1,1)
m.write('testcost.lp')

# Compute optimal solution
m.optimize()

#print solution 

for v in m.getVars():
    print(v.varName, v.x)
    
print('Obj:', m.objVal)    




NameError: name 'Eint' is not defined

In [10]:
#LOAD IMBALANCE WITH LINEAR INTERPOLATION
import numpy as np
from scipy import interpolate
from gurobipy import *
#import matplotlib.pyplot as plt
#def solve(SOCval):
m = Model("linearinterpLoadIm")
Eint = [2.0, 3.2] # Initial Energy in EV
Ereq = [-1.2, 4.6]
#te = [28.0, 24.0]

Energy = [-4.6,2.3,4.6]  #Energy in kwh
ImCost = [0,0,1]    #SOC level in %
f = interpolate.interp1d(Energy,ImCost)
x = Eint[1] #Energy30
y = Ereq[1] #R_E
Im_C = f(x)
R_E = f(y)
print(Im_C)
print(R_E)

0.39130434782608714
1.0


In [52]:
#!/usr/bin/python
import numpy as np
from scipy import interpolate
from gurobipy import *
m = Model("Generalizeparam")
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []
  
eff = 0.95
def parameter( M, T, Eint,Ereq,TS,TD):
    EI =[0 for t in range(len(Eint))]
    ER =[0 for t in range(len(Ereq))]
    for i in range(0,len(Eint)):
        EI[i] = Eint[i]
        ER[i] = Ereq[i]
    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
        #print("StartTime:" , TS[i])
        #print("DepartureTime:" , TD[i])
        #print("Initial energy:" , Eint[i])
    cost = [0.10, 0.20, 0.60, 0.40, 0.40, 0.30, 0.20, 0.20, 0.80, 0.60]
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    #Phase constraint
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
            
    #SOC user constraint
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            e[i][t] += p[i,t] * eff   # Energy consumed by EV1   ft *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]
            te[i] += e[i][t]
            n = te[i]
        SOC = [0,50,100]    #SOC level in %
        Energy = [0,20,40]  #Energy in kwh
        f = interpolate.interp1d(SOC,Energy)
        x = Eint[i] 
        y = Ereq[i] 
        I_E = f(x)
        R_E = f(y)
        I_E = int(np.round(I_E))
        R_E = int(np.round(R_E))
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
    
    #power outside TS TD as 0
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))

    #Grid constraint
    for t in range(T):
        for i in range(M):
    #if i < M :
            pa[i][t] = a[i,t] * p[i,t]
            pb[i][t] = b[i,t] * p[i,t]
            pc[i][t] = c[i,t] * p[i,t]
        #print(pa[i][t])
        patot[t] = pa[0][t] + pa[1][t]
        pbtot[t] = pb[0][t] + pb[1][t]
        pctot[t] = pc[0][t] + pc[1][t]
   
        pab[t] = patot[t] - pbtot[t]
        pbc[t] = pbtot[t] - pctot[t]
        pca[t] = pctot[t] - patot[t]
    
    
        m.addConstr(pab[t] <= 4.6, name="Power_A_B_constraint")
        m.addConstr(pab[t] >= -4.6, name="Power_A_B_constraint")
        m.addConstr(pbc[t] <= 4.6, name="Power_B_C_constraint")
        m.addConstr(pbc[t] >= -4.6, name="Power_B_C_constraint")
        m.addConstr(pca[t] <= 4.6, name="Power_C_A_constraint")
        m.addConstr(pca[t] >= -4.6, name="Power_C_A_constraint")
    
    m.update
    m.write('generalizeparam.lp')

    return;

Eint = [10,20];
Ereq = [80,60];
TS = [0, 4];       
TD = [6, 9];
parameter(2,96,Eint,Ereq,TS,TD)

StartTime: 0
DepartureTime: 6
StartTime: 4
DepartureTime: 9


In [30]:
#ALMOST FINAL
#!/usr/bin/python
import csv
import numpy as np
from scipy import interpolate
from gurobipy import *

eff = 15
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []

def parameterda( M, T, Eint,Ereq,TS,TD):
    EI =[0 for t in range(len(Eint))]
    ER =[0 for t in range(len(Ereq))]
    for i in range(0,len(Eint)):
        EI[i] = Eint[i]
        ER[i] = Ereq[i]
    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
        #print("StartTime:" , TS[i])
        #print("DepartureTime:" , TD[i])
        #print("Initial energy:" , Eint[i])
    #cost = [0.10, 0.20, 0.60, 0.40, 0.40, 0.30, 0.20, 0.20, 0.80, 0.60]
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/household8_profile.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHP_a = float(row[0])
        HHP_b = float(row[1])
        HHP_c = float(row[2])
        HHQ_a = float(row[3])
        HHQ_b = float(row[4])
        HHQ_c = float(row[5])
        HH.append([HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PV_P = float(row[0])
        PV.append(PV_P)
    m = Model("dayahead")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    #Phase constraint
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
            
    #SOC user constraint
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            e[i][t] +=  p[i,t] * eff * 1/4  # Energy consumed by EV1   1/4 *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]
            te[i] += e[i][t]
            n = te[i]
        SOC = [0,50,100]    #SOC level in %
        Energy = [0,20,40]  #Energy in kwh
        f = interpolate.interp1d(SOC,Energy)
        x = EI[i] 
        y = ER[i] 
        I_E = f(x)
        R_E = f(y)
        I_E = int(np.round(I_E))
        R_E = int(np.round(R_E))
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
    
    #power outside TS TD as 0
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))

    #Grid constraint
    for t in range(T):
        for i in range(M):
    #if i < M :
            pa[i][t] = a[i,t] * p[i,t]
            pb[i][t] = b[i,t] * p[i,t]
            pc[i][t] = c[i,t] * p[i,t]
            
            patot[t] += pa[i][t]
            pbtot[t] += pb[i][t]
            pctot[t] += pc[i][t]
        #print(pa[i][t])
        pahhpv[t] = patot[t] + HH[t][0] - PV[t]
        pbhhpv[t] = pbtot[t] + HH[t][1]
        pchhpv[t] = pctot[t] + HH[t][2]
   
        
        #pab[t] = patot[t] - pbtot[t]
        #pbc[t] = pbtot[t] - pctot[t]
        #pca[t] = pctot[t] - patot[t]
    
        pab[t] = pahhpv[t] - pbhhpv[t]
        pbc[t] = pbhhpv[t] - pchhpv[t]
        pca[t] = pchhpv[t] - pahhpv[t]
    
        m.addConstr(pab[t] <= 4.60, name="Power_A_B_maxconstraint")
        m.addConstr(pab[t] >= -4.60, name="Power_A_B_minconstraint")
        m.addConstr(pbc[t] <= 4.60, name="Power_B_C_maxconstraint")
        m.addConstr(pbc[t] >= -4.60, name="Power_B_C_minconstraint")
        m.addConstr(pca[t] <= 4.60, name="Power_C_A_maxconstraint")
        m.addConstr(pca[t] >= -4.60, name="Power_C_A_minconstraint")
    
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    Im_cost = C
    ST = STime[0]
    i = M - 1
    DT = DTime[i]
    for t in range(ST,DT):
        if pab[t] <= 2.3:
            Im_cost += 0.0
        elif pab[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pbc[t] <= 2.3:
            Im_cost += 0.0
        elif pbc[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pca[t] <= 2.3:
            Im_cost += 0.0
        elif pca[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
            
        #elif pab[t] <= -2.3:
        #    Im_cost += 1.0
        #elif pbc[t] >= 2.3:
        #    Im_cost += 1.0
        #elif pbc[t] <= -2.3:
        #    Im_cost += 1.0
        #elif pca[t] >= 2.3:
        #    Im_cost += 1.0
        #elif pca[t] <= -2.3:
        #    Im_cost += 1.0
    
    #set objective:
    m.ModelSense = GRB.MINIMIZE
    Im_C = Im_cost
    #m.setObjective(C)
    m.setObjective(Im_C)

    m.write('dayahead.lp')
    #m.write('testcost.lp')

    # Compute optimal solution
    m.optimize()
    m.write('dayahead.sol')
    #print solution 

    for v in m.getVars():
        print(v.varName, v.x)
    
    print('Obj:', m.objVal)
    
    with open('dayahead.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dayahead.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
# do IIS
#    print('The model is infeasible; computing IIS')
#    m.computeIIS()
#    if m.IISMinimal:
#      print('IIS is minimal\n')
#    else:
#      print('IIS is not minimal\n')
#    print('\nThe following constraint(s) cannot be satisfied:')
#    for c in m.getConstrs():
#        if c.IISConstr:
#             print('%s' % c.constrName)
    return;

Eint = [30,20];
Ereq = [80,80];
TS = [0, 5];       
TD = [12, 17];
parameterda(2,96,Eint,Ereq,TS,TD)

Optimize a model with 362 rows, 768 columns and 768 nonzeros
Model has 576 quadratic constraints
Variable types: 192 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [3e-01, 9e+00]
Presolve removed 336 rows and 672 columns
Presolve time: 0.02s
Presolved: 560 rows, 384 columns, 1680 nonzeros
Variable types: 312 continuous, 72 integer (72 binary)

Root relaxation: objective 1.229120e+01, 203 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   12.29120    0   26          -   12.29120      -     -    0s
H    0     0                      12.5242336   12.29120  1.86%     -    0s
H    0     0                      12.5074510   12.29120  1.73%     -    0s
H   

Power[1,22] 0.0
Power[1,23] 0.0
Power[1,24] 0.0
Power[1,25] 0.0
Power[1,26] 0.0
Power[1,27] 0.0
Power[1,28] 0.0
Power[1,29] 0.0
Power[1,30] 0.0
Power[1,31] 0.0
Power[1,32] 0.0
Power[1,33] 0.0
Power[1,34] 0.0
Power[1,35] 0.0
Power[1,36] 0.0
Power[1,37] 0.0
Power[1,38] 0.0
Power[1,39] 0.0
Power[1,40] 0.0
Power[1,41] 0.0
Power[1,42] 0.0
Power[1,43] 0.0
Power[1,44] 0.0
Power[1,45] 0.0
Power[1,46] 0.0
Power[1,47] 0.0
Power[1,48] 0.0
Power[1,49] 0.0
Power[1,50] 0.0
Power[1,51] 0.0
Power[1,52] 0.0
Power[1,53] 0.0
Power[1,54] 0.0
Power[1,55] 0.0
Power[1,56] 0.0
Power[1,57] 0.0
Power[1,58] 0.0
Power[1,59] 0.0
Power[1,60] 0.0
Power[1,61] 0.0
Power[1,62] 0.0
Power[1,63] 0.0
Power[1,64] 0.0
Power[1,65] 0.0
Power[1,66] 0.0
Power[1,67] 0.0
Power[1,68] 0.0
Power[1,69] 0.0
Power[1,70] 0.0
Power[1,71] 0.0
Power[1,72] 0.0
Power[1,73] 0.0
Power[1,74] 0.0
Power[1,75] 0.0
Power[1,76] 0.0
Power[1,77] 0.0
Power[1,78] 0.0
Power[1,79] 0.0
Power[1,80] 0.0
Power[1,81] 0.0
Power[1,82] 0.0
Power[1,83] 0.0
Power[1,

In [28]:
#Dumb charging
#!/usr/bin/python
import csv
import numpy as np
from scipy import interpolate
from gurobipy import *

eff = 15
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []

def parameterdc( M, T, Eint,Ereq,TS,TD):
    EI =[0 for t in range(len(Eint))]
    ER =[0 for t in range(len(Ereq))]
    for i in range(0,len(Eint)):
        EI[i] = Eint[i]
        ER[i] = Ereq[i]
    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
        #print("StartTime:" , TS[i])
        #print("DepartureTime:" , TD[i])
        #print("Initial energy:" , Eint[i])
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/household8_profile.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHP_a = float(row[0])
        HHP_b = float(row[1])
        HHP_c = float(row[2])
        HHQ_a = float(row[3])
        HHQ_b = float(row[4])
        HHQ_c = float(row[5])
        HH.append([HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PV_P = float(row[0])
        PV.append(PV_P)
    m = Model("Generalizeparam")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    
    #Phase constraint:
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
    
    #power outside TS TD as 0:
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
            
    #Subcost calculation:
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            e[i][t] +=  p[i,t] * eff * 1/4  # Energy consumed by EV1   1/4 *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]    
            te[i] += e[i][t]
            n = te[i]
        SOC = [0,50,100]    #SOC level in %
        Energy = [0,20,40]  #Energy in kwh
        f = interpolate.interp1d(SOC,Energy)
        x = EI[i] 
        y = ER[i] 
        I_E = f(x)
        R_E = f(y)
        I_E = int(np.round(I_E))
        R_E = int(np.round(R_E))
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
        
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    
    #set objective:
    m.ModelSense = GRB.MAXIMIZE
    m.setObjective(C)
    
    #writing the model:
    m.write('dumbcharging.lp')
    
    # Compute optimal solution:
    m.optimize()
    m.write('dumbcharge.sol')
    #print solution: 

    for v in m.getVars():
        print(v.varName, v.x)
    
    print('Obj:', m.objVal)
    
    with open('dumbcharge.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dumbcharge.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
    return;

Eint = [30,20];
Ereq = [80,80];
TS = [0, 5];       
TD = [12, 17];
parameterdc(2,96,Eint,Ereq,TS,TD)
    

# do IIS
#    print('The model is infeasible; computing IIS')
#    m.computeIIS()
#    if m.IISMinimal:
#      print('IIS is minimal\n')
#    else:
#      print('IIS is not minimal\n')
#    print('\nThe following constraint(s) cannot be satisfied:')
#    for c in m.getConstrs():
#        if c.IISConstr:
#             print('%s' % c.constrName)


Optimize a model with 362 rows, 768 columns and 768 nonzeros
Variable types: 192 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 12.5693988
Presolve removed 362 rows and 768 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 16 available processors)

Solution count 2: 13.1381 12.5694 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.313814740000e+01, best bound 1.313814740000e+01, gap 0.0000%
a[0,0] 1.0
a[0,1] 1.0
a[0,2] 1.0
a[0,3] 1.0
a[0,4] 1.0
a[0,5] 1.0
a[0,6] 1.0
a[0,7] 1.0
a[0,8] 1.0
a[0,9] 1.0
a[0,10] 1.0
a[0,11] 1.0
a[0,12] 1.0
a[0,13] 1.0
a[0,14] 1.0
a[0,15] 1.0
a[0,16] 1.0
a[0,17] 1.0
a[0,18] 1.0
a[0,19] 1.0
a[0,20] 1.0
a[0,21] 1.0
a[0,22] 1.0
a[0,23] 1.0
a[0,24] 1.0
a[0,25] 1.0
a[0,26]

Power[1,28] 0.0
Power[1,29] 0.0
Power[1,30] 0.0
Power[1,31] 0.0
Power[1,32] 0.0
Power[1,33] 0.0
Power[1,34] 0.0
Power[1,35] 0.0
Power[1,36] 0.0
Power[1,37] 0.0
Power[1,38] 0.0
Power[1,39] 0.0
Power[1,40] 0.0
Power[1,41] 0.0
Power[1,42] 0.0
Power[1,43] 0.0
Power[1,44] 0.0
Power[1,45] 0.0
Power[1,46] 0.0
Power[1,47] 0.0
Power[1,48] 0.0
Power[1,49] 0.0
Power[1,50] 0.0
Power[1,51] 0.0
Power[1,52] 0.0
Power[1,53] 0.0
Power[1,54] 0.0
Power[1,55] 0.0
Power[1,56] 0.0
Power[1,57] 0.0
Power[1,58] 0.0
Power[1,59] 0.0
Power[1,60] 0.0
Power[1,61] 0.0
Power[1,62] 0.0
Power[1,63] 0.0
Power[1,64] 0.0
Power[1,65] 0.0
Power[1,66] 0.0
Power[1,67] 0.0
Power[1,68] 0.0
Power[1,69] 0.0
Power[1,70] 0.0
Power[1,71] 0.0
Power[1,72] 0.0
Power[1,73] 0.0
Power[1,74] 0.0
Power[1,75] 0.0
Power[1,76] 0.0
Power[1,77] 0.0
Power[1,78] 0.0
Power[1,79] 0.0
Power[1,80] 0.0
Power[1,81] 0.0
Power[1,82] 0.0
Power[1,83] 0.0
Power[1,84] 0.0
Power[1,85] 0.0
Power[1,86] 0.0
Power[1,87] 0.0
Power[1,88] 0.0
Power[1,89] 0.0
Power[1,

In [35]:
import csv

# If you want to remove the first column and then every column after the fifth, keeping
# in mind that Python indexes start at 0

end = -1

with open("C:/Users/Subha Thillairajan/PycharmProjects/mosaik/Envs/mosaik/src/mosaik-powerfacotry/demos/data/household.csv", "r", newline='') as file_in:
    with open("C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH.csv", "w", newline='') as file_out:

        writer = csv.writer(file_out)

        for row in csv.reader(file_in):
            writer.writerow(row[:end])

In [8]:
import random
y =[]
with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/Arr_Dep_EV.csv') as f:
    y = print(random.choice(f.readlines()))
    #print(y[0])

25,19:30,21:25



In [ ]:
#Dynamic scheduling
#!/usr/bin/python
import csv
import numpy as np
import random
from scipy import interpolate
from gurobipy import *

eff = 15
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []

def parameterdy( M, T, Eint,Ereq,TS,TD):
    EI =[0 for t in range(len(Eint))]
    ER =[0 for t in range(len(Ereq))]
    for i in range(0,len(Eint)):
        EI[i] = Eint[i]
        ER[i] = Ereq[i]
    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
        #print("StartTime:" , TS[i])
        #print("DepartureTime:" , TD[i])
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/household8_profile.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHP_a = float(row[0])
        HHP_b = float(row[1])
        HHP_c = float(row[2])
        HHQ_a = float(row[3])
        HHQ_b = float(row[4])
        HHQ_c = float(row[5])
        HH.append([HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PV_P = float(row[0])
        PV.append(PV_P)
    m = Model("dayahead")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    #Phase constraint
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
            
    #SOC user constraint
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            e[i][t] +=  p[i,t] * eff * 1/4  # Energy consumed by EV1   1/4 *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]
            te[i] += e[i][t]
            n = te[i]
        SOC = [0,50,100]    #SOC level in %
        Energy = [0,20,40]  #Energy in kwh
        f = interpolate.interp1d(SOC,Energy)
        x = EI[i] 
        y = ER[i] 
        I_E = f(x)
        R_E = f(y)
        I_E = int(np.round(I_E))
        R_E = int(np.round(R_E))
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
    
    #power outside TS TD as 0
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))

    #Grid constraint
    for t in range(T):
        for i in range(M):
    #if i < M :
            pa[i][t] = a[i,t] * p[i,t]
            pb[i][t] = b[i,t] * p[i,t]
            pc[i][t] = c[i,t] * p[i,t]
            
            patot[t] += pa[i][t]
            pbtot[t] += pb[i][t]
            pctot[t] += pc[i][t]
        #print(pa[i][t])
        pahhpv[t] = patot[t] + HH[t][0] - PV[t]
        pbhhpv[t] = pbtot[t] + HH[t][1]
        pchhpv[t] = pctot[t] + HH[t][2]
   
        
        #pab[t] = patot[t] - pbtot[t]
        #pbc[t] = pbtot[t] - pctot[t]
        #pca[t] = pctot[t] - patot[t]
    
        pab[t] = pahhpv[t] - pbhhpv[t]
        pbc[t] = pbhhpv[t] - pchhpv[t]
        pca[t] = pchhpv[t] - pahhpv[t]
    
        m.addConstr(pab[t] <= 4.60, name="Power_A_B_maxconstraint")
        m.addConstr(pab[t] >= -4.60, name="Power_A_B_minconstraint")
        m.addConstr(pbc[t] <= 4.60, name="Power_B_C_maxconstraint")
        m.addConstr(pbc[t] >= -4.60, name="Power_B_C_minconstraint")
        m.addConstr(pca[t] <= 4.60, name="Power_C_A_maxconstraint")
        m.addConstr(pca[t] >= -4.60, name="Power_C_A_minconstraint")
    
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    Im_cost = C
    ST = STime[0]
    i = M - 1
    DT = DTime[i]
    for t in range(ST,DT):
        if pab[t] <= 2.3:
            Im_cost += 0.0
        elif pab[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pbc[t] <= 2.3:
            Im_cost += 0.0
        elif pbc[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pca[t] <= 2.3:
            Im_cost += 0.0
        elif pca[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
            
        #elif pab[t] <= -2.3:
        #    Im_cost += 1.0
        #elif pbc[t] >= 2.3:
        #    Im_cost += 1.0
        #elif pbc[t] <= -2.3:
        #    Im_cost += 1.0
        #elif pca[t] >= 2.3:
        #    Im_cost += 1.0
        #elif pca[t] <= -2.3:
        #    Im_cost += 1.0
    
    #set objective:
    m.ModelSense = GRB.MINIMIZE
    Im_C = Im_cost
    #m.setObjective(C)
    m.setObjective(Im_C)

    m.write('Dynamicscheduling.lp')
    #m.write('testcost.lp')

    # Compute optimal solution
    m.optimize()
    m.write('Dynamicscheduling.sol')
    #print solution 

    for v in m.getVars():
        print(v.varName, v.x)
    
    print('Obj:', m.objVal)
    
    with open('Dynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('Dynamicscheduling.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
# do IIS
#    print('The model is infeasible; computing IIS')
#    m.computeIIS()
#    if m.IISMinimal:
#      print('IIS is minimal\n')
#    else:
#      print('IIS is not minimal\n')
#    print('\nThe following constraint(s) cannot be satisfied:')
#    for c in m.getConstrs():
#        if c.IISConstr:
#             print('%s' % c.constrName)
    return;
with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/Arr_Dep_EV.csv') as f:
    
    print(random.choice(f.readlines()))
Eint = [30,20];
Ereq = [80,80];
TS = [0, 5];       
TD = [12, 17];
parameterdy(2,96,Eint,Ereq,TS,TD)

In [3]:
#dayahead scheduling
#!/usr/bin/python
import csv
import numpy as np
from datetime import time,datetime,date
from scipy import interpolate
from gurobipy import *

eff = 15
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []
thisdict = {
  '01.01.2014 00:00:00': 0,
  '01.01.2014 00:15:00': 1,
  '01.01.2014 00:30:00': 2
}
thisdict.update([ ('01.01.2014 00:45:00', 3) , ('01.01.2014 01:00:00', 4) , ('01.01.2014 01:15:00', 5) ,
                 ('01.01.2014 01:30:00', 6),('01.01.2014 01:45:00', 7),('01.01.2014 02:00:00', 8),
                 ('01.01.2014 02:15:00', 9),('01.01.2014 02:30:00', 10),('01.01.2014 02:45:00', 11),
                 ('01.01.2014 03:00:00', 12),('01.01.2014 03:15:00', 13),('01.01.2014 03:30:00', 14) , 
                 ('01.01.2014 03:45:00', 15) , ('01.01.2014 04:00:00', 16) , ('01.01.2014 04:15:00', 17),
                 ('01.01.2014 04:30:00', 18),('01.01.2014 04:45:00', 19),('01.01.2014 05:00:00', 20),
                 ('01.01.2014 05:15:00', 21),('01.01.2014 05:30:00', 22),('01.01.2014 05:45:00', 23),
                 ('01.01.2014 06:00:00', 24),('01.01.2014 06:15:00', 25) , ('01.01.2014 06:30:00', 26),
                 ('01.01.2014 06:45:00', 27),('01.01.2014 07:00:00', 28),('01.01.2014 07:15:00', 29),
                 ('01.01.2014 07:30:00', 30),('01.01.2014 07:45:00', 31),('01.01.2014 08:00:00', 32),
                 ('01.01.2014 08:15:00', 33),('01.01.2014 08:30:00', 34) , ('01.01.2014 08:45:00', 35) , 
                 ('01.01.2014 09:00:00', 36) , ('01.01.2014 09:15:00', 37),('01.01.2014 09:30:00', 38),
                 ('01.01.2014 09:45:00', 39),('01.01.2014 10:00:00', 40),('01.01.2014 10:15:00', 41),
                 ('01.01.2014 10:30:00', 42),('01.01.2014 10:45:00', 43),('01.01.2014 11:00:00', 44),
                 ('01.01.2014 11:15:00', 45) , ('01.01.2014 11:30:00', 46),('01.01.2014 11:45:00', 47),
                 ('01.01.2014 12:00:00', 48),('01.01.2014 12:15:00', 49),('01.01.2014 12:30:00', 50),
                 ('01.01.2014 12:45:00', 51),('01.01.2014 13:00:00', 52),('01.01.2014 13:15:00', 53),
                 ('01.01.2014 13:30:00', 54) , ('01.01.2014 13:45:00', 55) , ('01.01.2014 14:00:00', 56) , 
                 ('01.01.2014 14:15:00', 57),('01.01.2014 14:30:00', 58),('01.01.2014 14:45:00', 59),
                 ('01.01.2014 15:00:00', 60),('01.01.2014 15:15:00', 61),('01.01.2014 15:30:00', 62),
                 ('01.01.2014 15:45:00', 63),('01.01.2014 16:00:00', 64),('01.01.2014 16:15:00', 65) , 
                 ('01.01.2014 16:30:00', 66),('01.01.2014 16:45:00', 67),('01.01.2014 17:00:00', 68),
                 ('01.01.2014 17:15:00', 69),('01.01.2014 17:30:00', 70),('01.01.2014 17:45:00', 71),
                 ('01.01.2014 18:00:00', 72),('01.01.2014 18:15:00', 73),('01.01.2014 18:30:00', 74) , 
                 ('01.01.2014 18:45:00', 75) , ('01.01.2014 19:00:00', 76) , ('01.01.2014 19:15:00', 77),
                 ('01.01.2014 19:30:00', 78),('01.01.2014 19:45:00', 79),('01.01.2014 20:00:00', 80),
                 ('01.01.2014 20:15:00', 81),('01.01.2014 20:30:00', 82),('01.01.2014 20:45:00', 83),
                 ('01.01.2014 21:00:00',84),('01.01.2014 21:15:00', 85) , ('01.01.2014 21:30:00', 86),
                 ('01.01.2014 21:45:00', 87),('01.01.2014 22:00:00', 88),('01.01.2014 22:15:00', 89),
                 ('01.01.2014 22:30:00', 90),('01.01.2014 00:45:00', 91),('01.01.2014 23:00:00', 92),
                 ('01.01.2014 23:15:00', 93),('01.01.2014 23:30:00', 94) , ('01.01.2014 23:45:00', 95)])
def parameterda( M, T, Eint,Ereq,TS,TD):
    EI =[0 for t in range(len(Eint))]
    ER =[0 for t in range(len(Ereq))]
    for i in range(0,len(Eint)):
        EI[i] = Eint[i]
        ER[i] = Ereq[i]
    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH_profile_Scheduling.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHDate = row[0]
        HHP_a = float(row[1])
        HHP_b = float(row[2])
        HHP_c = float(row[3])
        HHQ_a = float(row[4])
        HHQ_b = float(row[5])
        HHQ_c = float(row[6])
        HH.append([HHDate,HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile_Scheduling.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PVDate = row[0]
        PV_P = float(row[1])
        PV.append([PVDate,PV_P])
    m = Model("dayahead")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    #Phase constraint
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
            
    #SOC user constraint
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            e[i][t] +=  p[i,t] * eff * 1/4  # Energy consumed by EV1   1/4 *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]
            te[i] += e[i][t]
            n = te[i]
        SOC = [0,50,100]    #SOC level in %
        Energy = [0,20,40]  #Energy in kwh
        f = interpolate.interp1d(SOC,Energy)
        x = EI[i] 
        y = ER[i] 
        I_E = f(x)
        R_E = f(y)
        I_E = int(np.round(I_E))
        R_E = int(np.round(R_E))
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
    
    #power outside TS TD as 0
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))

    #Grid constraint
    for t in range(T):
        #if HH[i][0] == atime:
        for i in range(M):
    #if i < M :
            pa[i][t] = a[i,t] * p[i,t]
            pb[i][t] = b[i,t] * p[i,t]
            pc[i][t] = c[i,t] * p[i,t]
            
            patot[t] += pa[i][t]
            pbtot[t] += pb[i][t]
            pctot[t] += pc[i][t]
        #print(pa[i][t])
        pahhpv[t] = patot[t] + HH[t][1] - PV[t][1]
        pbhhpv[t] = pbtot[t] + HH[t][2]
        pchhpv[t] = pctot[t] + HH[t][3]
   
    
        pab[t] = pahhpv[t] - pbhhpv[t]
        pbc[t] = pbhhpv[t] - pchhpv[t]
        pca[t] = pchhpv[t] - pahhpv[t]
    
        m.addConstr(pab[t] <= 4.60, name="Power_A_B_maxconstraint")
        m.addConstr(pab[t] >= -4.60, name="Power_A_B_minconstraint")
        m.addConstr(pbc[t] <= 4.60, name="Power_B_C_maxconstraint")
        m.addConstr(pbc[t] >= -4.60, name="Power_B_C_minconstraint")
        m.addConstr(pca[t] <= 4.60, name="Power_C_A_maxconstraint")
        m.addConstr(pca[t] >= -4.60, name="Power_C_A_minconstraint")
    
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    Im_cost = C
    ST = STime[0]
    i = M - 1
    DT = DTime[i]
    for t in range(ST,DT):
        if pab[t] <= 2.3:
            Im_cost += 0.0
        elif pab[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pbc[t] <= 2.3:
            Im_cost += 0.0
        elif pbc[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pca[t] <= 2.3:
            Im_cost += 0.0
        elif pca[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
    
    #set objective:
    m.ModelSense = GRB.MINIMIZE
    Im_C = Im_cost
    #m.setObjective(C)
    m.setObjective(Im_C)

    m.write('dayahead.lp')
    #m.write('testcost.lp')

    # Compute optimal solution
    m.optimize()
    m.write('dayahead.sol')
    #print solution 

    for v in m.getVars():
        print(v.varName, v.x)
    
    print('Obj:', m.objVal)
    
    with open('dayahead.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dayahead.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
# do IIS
#    print('The model is infeasible; computing IIS')
#    m.computeIIS()
#    if m.IISMinimal:
#      print('IIS is minimal\n')
#    else:
#      print('IIS is not minimal\n')
#    print('\nThe following constraint(s) cannot be satisfied:')
#    for c in m.getConstrs():
#        if c.IISConstr:
#             print('%s' % c.constrName)
    return;

Eint = [30,20];
Ereq = [80,80];
#atime =[0 for t in range(M)]
#dtime =[0 for t in range(M)]
EV1_a = datetime(2014, 1, 1, 16, 30)
atime1 = EV1_a.strftime("%d.%m.%Y %H:%M:%S")
EV2_a = datetime(2014, 1, 1, 18, 15)
atime2 = EV2_a.strftime("%d.%m.%Y %H:%M:%S")
EV1_d = datetime(2014, 1, 1, 22, 15)
dtime1 = EV1_d.strftime("%d.%m.%Y %H:%M:%S")
EV2_d = datetime(2014, 1, 1, 23, 45)
dtime2 = EV2_d.strftime("%d.%m.%Y %H:%M:%S")
for item,val in thisdict.items():
    if item == atime1:
        ev1_a = val
    elif item == atime2:
        ev2_a = val
    elif item == dtime1:
        ev1_d = val    
    elif item == dtime2:
        ev2_d = val        
TS = [ev1_a,ev2_a]
TD = [ev1_d,ev2_d]
parameterda(2,96,Eint,Ereq,TS,TD)

Optimize a model with 341 rows, 768 columns and 768 nonzeros
Model has 576 quadratic constraints
Variable types: 192 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [3e-01, 9e+00]
Presolve removed 294 rows and 588 columns
Presolve time: 0.01s
Presolved: 1031 rows, 720 columns, 3150 nonzeros
Variable types: 585 continuous, 135 integer (135 binary)

Root relaxation: objective 7.246100e+00, 399 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    7.24610    0   73          -    7.24610      -     -    0s
H    0     0                      10.3770015    7.24610  30.2%     -    0s
H    0     0                      10.2991133    7.24610  29.6%     -    0s
H

 700074 201220    9.40911   38   42    9.41904    9.26280  1.66%  17.0  330s
 711870 203672    9.40863   47   31    9.41904    9.26386  1.65%  17.0  335s
 721594 205371     cutoff   47         9.41904    9.26470  1.64%  17.0  340s
 731946 207081    9.39562   48   32    9.41904    9.26589  1.63%  17.0  345s
 742150 208763     cutoff   40         9.41904    9.26680  1.62%  17.1  350s
 752236 210147    9.38180   43   37    9.41904    9.26770  1.61%  17.1  355s
 761867 211655     cutoff   50         9.41904    9.26874  1.60%  17.1  360s
 773692 213552    9.37662   49   34    9.41904    9.26994  1.58%  17.1  365s
 783571 214902    9.34331   41   41    9.41904    9.27084  1.57%  17.1  370s
 793365 216325     cutoff   40         9.41904    9.27186  1.56%  17.1  375s
 802976 217703    9.41174   45   32    9.41904    9.27281  1.55%  17.1  380s
 812759 219070    9.39577   45   37    9.41904    9.27381  1.54%  17.2  385s
 824462 220640     cutoff   46         9.41904    9.27466  1.53%  17.2  390s

 1866746 419358    9.36984   49   37    9.41904    9.33015  0.94%  16.7  860s
 1883887 421019    9.41297   54   22    9.41904    9.33054  0.94%  16.7  865s
 1901335 423179    9.41884   52   27    9.41904    9.33097  0.94%  16.7  870s
 1918656 424973     cutoff   60         9.41904    9.33129  0.93%  16.7  875s
 1936157 426512    9.39802   50   32    9.41904    9.33178  0.93%  16.6  880s
 1947922 427499    9.40777   48   25    9.41904    9.33211  0.92%  16.6  885s
 1967040 428777    9.41816   51   30    9.41904    9.33256  0.92%  16.6  890s
 1986695 430860    9.37959   47   29    9.41904    9.33304  0.91%  16.6  895s
 2008128 432555     cutoff   46         9.41904    9.33368  0.91%  16.6  900s
 2025217 433525    9.41450   52   20    9.41904    9.33407  0.90%  16.6  905s
 2047714 434797    9.40438   41   35    9.41904    9.33463  0.90%  16.6  910s
 2071563 437685    9.37870   46   34    9.41904    9.33516  0.89%  16.6  915s
 2092421 438898    9.38040   48   32    9.41904    9.33561  0.89

 4312319 577384     cutoff   51         9.41904    9.37035  0.52%  16.8 1390s
 4337349 577388    9.37161   44   39    9.41904    9.37068  0.51%  16.8 1395s
 4362869 577947     cutoff   58         9.41904    9.37097  0.51%  16.8 1400s
 4381830 577692    9.38228   44   36    9.41904    9.37129  0.51%  16.8 1405s
 4398965 576932    9.41634   46   33    9.41904    9.37155  0.50%  16.8 1410s
 4420675 576752     cutoff   46         9.41904    9.37182  0.50%  16.8 1415s
 4443510 576425    9.41850   52   21    9.41904    9.37216  0.50%  16.8 1420s
 4464969 575760    9.41530   47   31    9.41904    9.37251  0.49%  16.8 1425s
 4486308 574728    9.41672   49   26    9.41904    9.37284  0.49%  16.8 1430s
 4509676 573798    9.41853   49   24    9.41904    9.37324  0.49%  16.8 1435s
 4530301 573114     cutoff   51         9.41904    9.37356  0.48%  16.8 1440s
 4551104 571681    9.40988   47   28    9.41904    9.37384  0.48%  16.8 1445s
 4572180 570382    9.41761   40   32    9.41904    9.37418  0.48

 6193335 754963    9.39514   51   27    9.41904    9.39215  0.29%  16.3 1920s
 6210366 756171    9.41784   57    6    9.41904    9.39227  0.28%  16.3 1925s
 6228104 758968    9.41890   52   12    9.41904    9.39233  0.28%  16.3 1930s
 6243910 761904    9.40454   50   19    9.41904    9.39239  0.28%  16.3 1936s
 6255182 763899    9.39319   52   25    9.41904    9.39243  0.28%  16.3 1940s
 6273155 765800    9.41898   57   14    9.41904    9.39252  0.28%  16.3 1945s
 6288811 767623    9.39477   55   22    9.41904    9.39258  0.28%  16.3 1950s
 6305924 770357     cutoff   53         9.41904    9.39265  0.28%  16.3 1955s
 6321690 774028     cutoff   58         9.41904    9.39265  0.28%  16.3 1960s
 6338707 777058    9.40455   50   20    9.41904    9.39269  0.28%  16.3 1965s
 6353352 779037    9.41499   45   34    9.41904    9.39276  0.28%  16.3 1970s
 6369443 781570    9.40409   50   19    9.41904    9.39281  0.28%  16.3 1975s
 6384692 783819    9.41838   57    8    9.41904    9.39285  0.28

...

 11371262 929738    9.41732   52   22    9.41904    9.40542  0.14%  15.4 3500s
 11383781 929378    9.40546   54   15    9.41904    9.40544  0.14%  15.4 3505s
 11395272 928673     cutoff   59         9.41904    9.40545  0.14%  15.4 3510s
 11406870 928643    9.41810   53   20    9.41904    9.40546  0.14%  15.4 3515s
 11418497 928528    9.41763   54   10    9.41904    9.40549  0.14%  15.4 3520s
 11429967 928385    9.40869   52   23    9.41904    9.40550  0.14%  15.4 3525s
 11442104 927253    9.41429   53   15    9.41904    9.40552  0.14%  15.4 3530s
 11454059 926573    9.41848   53   23    9.41904    9.40553  0.14%  15.4 3535s
 11464789 926114    9.41810   57   17    9.41904    9.40555  0.14%  15.4 3540s
 11476894 925878     cutoff   53         9.41904    9.40557  0.14%  15.4 3545s
 11487318 925341     cutoff   59         9.41904    9.40558  0.14%  15.4 3550s
 11499290 924646     cutoff   54         9.41904    9.40560  0.14%  15.3 3555s
 11509062 924093     cutoff   60         9.41904    

 12486430 848108    9.41869   56   19    9.41904    9.40740  0.12%  15.2 4020s
 12495348 846690    9.41040   50   25    9.41904    9.40741  0.12%  15.2 4025s
 12500644 846040     cutoff   55         9.41904    9.40743  0.12%  15.2 4030s
 12508297 845244    9.40810   54   15    9.41904    9.40744  0.12%  15.2 4035s
 12515918 843568     cutoff   58         9.41904    9.40746  0.12%  15.2 4040s
 12525071 841919    9.40841   54   10    9.41904    9.40749  0.12%  15.2 4045s
 12533806 840374    9.40819   56   14    9.41904    9.40753  0.12%  15.2 4050s
 12543270 838684    9.40821   54   16    9.41904    9.40755  0.12%  15.2 4055s
 12553035 837268     cutoff   56         9.41904    9.40759  0.12%  15.2 4060s
 12563499 836139    9.41068   53   21    9.41904    9.40762  0.12%  15.2 4065s
 12572688 835457     cutoff   62         9.41904    9.40763  0.12%  15.2 4070s
 12582513 834155    9.41823   55   11    9.41904    9.40767  0.12%  15.2 4075s
 12592431 832894    9.41069   50   23    9.41904    

 13445214 695722    9.41037   48   25    9.41904    9.41001  0.10%  15.2 4540s
 13455855 693605    9.41103   52   19    9.41904    9.41006  0.10%  15.1 4545s
 13465197 691253     cutoff   56         9.41904    9.41010  0.09%  15.1 4550s
 13474103 688719    9.41085   47   25    9.41904    9.41014  0.09%  15.1 4555s
 13482521 687073    9.41348   58    8    9.41904    9.41016  0.09%  15.1 4560s
 13491740 684264     cutoff   44         9.41904    9.41019  0.09%  15.1 4565s
 13500823 682286    9.41059   49   25    9.41904    9.41022  0.09%  15.1 4570s
 13511302 679543    9.41062   52   25    9.41904    9.41026  0.09%  15.1 4575s
 13519255 677331    9.41094   49   22    9.41904    9.41028  0.09%  15.1 4580s
 13529583 674571     cutoff   57         9.41904    9.41032  0.09%  15.1 4585s
 13538697 672401     cutoff   52         9.41904    9.41035  0.09%  15.1 4590s
 13548969 669733     cutoff   47         9.41904    9.41040  0.09%  15.1 4595s
 13556447 668617    9.41285   50   22    9.41904    

 14447707 442265    9.41369   59   17    9.41904    9.41363  0.06%  15.1 5060s
 14457718 439155    9.41442   60   12    9.41904    9.41368  0.06%  15.1 5065s
 14467005 435995    9.41398   55    6    9.41904    9.41374  0.06%  15.1 5070s
 14476952 433148    9.41377   56   22    9.41904    9.41377  0.06%  15.1 5075s
 14488452 430283    9.41454   57   18    9.41904    9.41380  0.06%  15.1 5080s
 14497787 428385    9.41479   54   25    9.41904    9.41383  0.06%  15.1 5085s
 14510001 424542     cutoff   55         9.41904    9.41387  0.05%  15.1 5090s
 14521420 421891    9.41390   63   13    9.41904    9.41390  0.05%  15.1 5095s
 14531136 418585     cutoff   51         9.41904    9.41392  0.05%  15.1 5100s
 14541924 414397     cutoff   51         9.41904    9.41397  0.05%  15.1 5105s
 14552768 410583    9.41500   53   18    9.41904    9.41404  0.05%  15.1 5110s
 14557751 408785    9.41472   50   18    9.41904    9.41406  0.05%  15.1 5115s
 14567968 404959    9.41508   58   14    9.41904    

 15484893 119776     cutoff   55         9.41904    9.41722  0.02%  15.0 5580s
 15493967 115758    9.41885   57   12    9.41904    9.41726  0.02%  15.0 5585s
 15504341 111533     cutoff   57         9.41904    9.41732  0.02%  15.0 5590s
 15516260 105045    9.41754   60    9    9.41904    9.41741  0.02%  15.0 5595s
 15524760 103883     cutoff   55         9.41904    9.41742  0.02%  15.0 5600s
 15531445 100511    9.41782   52   16    9.41904    9.41748  0.02%  15.0 5605s
 15543505 98026    9.41774   56    6    9.41904    9.41750  0.02%  15.0 5610s
 15552314 94951    9.41796   56    9    9.41904    9.41754  0.02%  15.0 5615s
 15561206 90239     cutoff   56         9.41904    9.41763  0.02%  15.0 5620s
 15572781 82517    9.41798   57    6    9.41904    9.41774  0.01%  15.0 5625s
 15581829 77479     cutoff   53         9.41904    9.41782  0.01%  15.0 5630s
 15592125 73502     cutoff   58         9.41904    9.41784  0.01%  15.0 5636s
 15601125 69155     cutoff   54         9.41904    9.41790

In [87]:
#Time function
import csv
from datetime import time,datetime,date
d = datetime(2014, 1, 1, 0, 0)
atime = d.strftime("%d.%m.%Y %H:%M:%S")

#atime = datetime.combine(D, t)
#d.strftime("%d.%m.%y")
print(atime)

path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH_profile_Scheduling.csv"
file = open(path1, newline='')
reader = csv.reader(file)
header = next(reader)
#rows = [header] + [[row[0], float(row[1])] for row in reader]
#for row in rows:
#    print(row)
HH = []
for row in reader :
    HHDate = row[0]
    HHP_a = float(row[1])
    HHP_b = float(row[2])
    HHP_c = float(row[3])
    HHQ_a = float(row[4])
    HHQ_b = float(row[5])
    HHQ_c = float(row[6])
    HH.append([HHDate,HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
#print(header)
#print(len(HH))
for i in range(len(HH)):
    if HH[i][0] == atime:
        print(HH[i])
    #print(HH[0])
print(HH[48][0])
#file.readlines(300)
#reader = csv.reader(file)
#header = next(reader)
#for i in reader:
    #print(HH[1][0])
    #print(reader[i])


01.01.2014 00:00:00
['01.01.2014 00:00:00', 0.104, 0.088, 0.391, 0.0, -0.008, 0.021]
01.01.2014 12:00:00


In [61]:
from datetime import time,datetime,date
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []
def parameterda( M, T, Eint,Ereq,TS,TD):
    EI =[0 for t in range(len(Eint))]
    ER =[0 for t in range(len(Ereq))]
    for i in range(0,len(Eint)):
        EI[i] = Eint[i]
        ER[i] = Ereq[i]
    STime = [0 for t in range(M)]
    DTime = [0 for t in range(M)]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
        print(STime[i])
        print(DTime[i])
Eint = [30,20];
Ereq = [80,80];
atime =[0 for t in range(M)]
dtime =[0 for t in range(M)]
EV1_a = datetime(2014, 1, 1, 16, 30)
atime[0] = EV1_a.strftime("%d.%m.%Y %H:%M:%S")
EV2_a = datetime(2014, 1, 1, 18, 15)
atime[1] = EV2_a.strftime("%d.%m.%Y %H:%M:%S")
EV1_d = datetime(2014, 1, 1, 22, 15)
dtime[0] = EV1_d.strftime("%d.%m.%Y %H:%M:%S")
EV2_d = datetime(2014, 1, 1, 23, 45)
dtime[1] = EV2_d.strftime("%d.%m.%Y %H:%M:%S")
TS = [atime[0],atime[1]]
TD = [dtime[0],dtime[1]]
parameterda(2,96,Eint,Ereq,TS,TD)

01.01.2014 16:30:00
01.01.2014 22:15:00
01.01.2014 18:15:00
01.01.2014 23:45:00


In [27]:
from datetime import time,datetime,date
import csv
T = 96
EV1_a = datetime(2014, 1, 1, 16, 30)
atime = EV1_a.strftime("%d.%m.%Y %H:%M:%S")
#print(atime)
path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH_profile_Scheduling.csv"
file = open(path1, newline='')
reader = csv.reader(file)
header = next(reader)
HH = []
for row in reader :
    HHDate = row[0]
    HHP_a = float(row[1])
    HHP_b = float(row[2])
    HHP_c = float(row[3])
    HHQ_a = float(row[4])
    HHQ_b = float(row[5])
    HHQ_c = float(row[6])
    HH.append([HHDate,HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
thisdict = {
  '01.01.2014 00:00:00': 0,
  '01.01.2014 00:15:00': 1,
  '01.01.2014 00:30:00': 2
}
thisdict.update([ ('01.01.2014 00:45:00', 3) , ('01.01.2014 01:00:00', 4) , ('01.01.2014 01:15:00', 5) ,
                 ('01.01.2014 01:30:00', 6),('01.01.2014 01:45:00', 7),('01.01.2014 02:00:00', 8),
                 ('01.01.2014 02:15:00', 9),('01.01.2014 02:30:00', 10),('01.01.2014 02:45:00', 11),
                 ('01.01.2014 03:00:00', 12),('01.01.2014 03:15:00', 13),('01.01.2014 03:30:00', 14) , 
                 ('01.01.2014 03:45:00', 15) , ('01.01.2014 04:00:00', 16) , ('01.01.2014 04:15:00', 17),
                 ('01.01.2014 04:30:00', 18),('01.01.2014 04:45:00', 19),('01.01.2014 05:00:00', 20),
                 ('01.01.2014 05:15:00', 21),('01.01.2014 05:30:00', 22),('01.01.2014 05:45:00', 23),
                 ('01.01.2014 06:00:00', 24),('01.01.2014 06:15:00', 25) , ('01.01.2014 06:30:00', 26),
                 ('01.01.2014 06:45:00', 27),('01.01.2014 07:00:00', 28),('01.01.2014 07:15:00', 29),
                 ('01.01.2014 07:30:00', 30),('01.01.2014 07:45:00', 31),('01.01.2014 08:00:00', 32),
                 ('01.01.2014 08:15:00', 33),('01.01.2014 08:30:00', 34) , ('01.01.2014 08:45:00', 35) , 
                 ('01.01.2014 09:00:00', 36) , ('01.01.2014 09:15:00', 37),('01.01.2014 09:30:00', 38),
                 ('01.01.2014 09:45:00', 39),('01.01.2014 10:00:00', 40),('01.01.2014 10:15:00', 41),
                 ('01.01.2014 10:30:00', 42),('01.01.2014 10:45:00', 43),('01.01.2014 11:00:00', 44),
                 ('01.01.2014 11:15:00', 45) , ('01.01.2014 11:30:00', 46),('01.01.2014 11:45:00', 47),
                 ('01.01.2014 12:00:00', 48),('01.01.2014 12:15:00', 49),('01.01.2014 12:30:00', 50),
                 ('01.01.2014 12:45:00', 51),('01.01.2014 13:00:00', 52),('01.01.2014 13:15:00', 53),
                 ('01.01.2014 13:30:00', 54) , ('01.01.2014 13:45:00', 55) , ('01.01.2014 14:00:00', 56) , 
                 ('01.01.2014 14:15:00', 57),('01.01.2014 14:30:00', 58),('01.01.2014 14:45:00', 59),
                 ('01.01.2014 15:00:00', 60),('01.01.2014 15:15:00', 61),('01.01.2014 15:30:00', 62),
                 ('01.01.2014 15:45:00', 63),('01.01.2014 16:00:00', 64),('01.01.2014 16:15:00', 65) , 
                 ('01.01.2014 16:30:00', 66),('01.01.2014 16:45:00', 67),('01.01.2014 17:00:00', 68),
                 ('01.01.2014 17:15:00', 69),('01.01.2014 17:30:00', 70),('01.01.2014 17:45:00', 71),
                 ('01.01.2014 18:00:00', 72),('01.01.2014 18:15:00', 73),('01.01.2014 18:30:00', 74) , 
                 ('01.01.2014 18:45:00', 75) , ('01.01.2014 19:00:00', 76) , ('01.01.2014 19:15:00', 77),
                 ('01.01.2014 19:30:00', 78),('01.01.2014 19:45:00', 79),('01.01.2014 20:00:00', 80),
                 ('01.01.2014 20:15:00', 81),('01.01.2014 20:30:00', 82),('01.01.2014 20:45:00', 83),
                 ('01.01.2014 21:00:00',84),('01.01.2014 21:15:00', 85) , ('01.01.2014 21:30:00', 86),
                 ('01.01.2014 21:45:00', 87),('01.01.2014 22:00:00', 88),('01.01.2014 22:15:00', 89),
                 ('01.01.2014 22:30:00', 90),('01.01.2014 00:45:00', 91),('01.01.2014 23:00:00', 92),
                 ('01.01.2014 23:15:00', 93),('01.01.2014 23:30:00', 94) , ('01.01.2014 23:45:00', 95)]) 
for item,val in thisdict.items():
    if item == atime:
        x=val
        print(x)
#print(thisdict)
#x=thisdict[HH[1][0]]
#print(x)
#for i in range(0,T):   
#    if thisdict[HH[i][0]] == atime:
#        y=thisdict[HH[i][0]]
#        print(y)

66


In [4]:
import csv
with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH.txt', newline='') as f:
    reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
    #next(reader)
    #next(reader)
    lines = list(reader)

with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH.csv', 'w', newline='') as myfile:
    wr = csv.writer(myfile, delimiter=',')
    wr.writerows(lines)
    
    f.close()
    myfile.close()